In [1]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.3.1


In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28*28) / 255.0
test_images = test_images[:1000].reshape(-1, 28*28) / 255.0

11493376/11490434 [==============================] - ETA:  - 0s 0us/step


In [3]:
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512, activation='relu', input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10)
    ])
    
    model.compile(optimizer='adam',
                  loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=[tf.metrics.SparseCategoricalAccuracy()])
    
    return model

model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [4]:
checkpoint_path = 'training_1/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath = checkpoint_path,
                                                 save_weights_only = True,
                                                 verbose = 1)

model.fit(train_images,
          train_labels,
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])

Epoch 1/10
 1/32 [..............................] - ETA: 0s - loss: 2.4095 - sparse_categorical_accuracy: 0.0938
Epoch 00001: saving model to training_1\cp.ckpt
32/32 [==============================] - 0s 7ms/step - loss: 1.1471 - sparse_categorical_accuracy: 0.6820 - val_loss: 0.7176 - val_sparse_categorical_accuracy: 0.7710
Epoch 2/10
31/32 [============================>.] - ETA: 0s - loss: 0.4103 - sparse_categorical_accuracy: 0.8931
Epoch 00002: saving model to training_1\cp.ckpt
32/32 [==============================] - 0s 4ms/step - loss: 0.4091 - sparse_categorical_accuracy: 0.8940 - val_loss: 0.5241 - val_sparse_categorical_accuracy: 0.8360
Epoch 3/10
 1/32 [..............................] - ETA: 0s - loss: 0.3004 - sparse_categorical_accuracy: 0.9375
Epoch 00003: saving model to training_1\cp.ckpt
32/32 [==============================] - 0s 4ms/step - loss: 0.2672 - sparse_categorical_accuracy: 0.9320 - val_loss: 0.4680 - val_sparse_categorical_accuracy: 0.8560
Epoch 4/10
29/32

In [7]:
model = create_model()

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 2.3657 - sparse_categorical_accuracy: 0.0710
Untrained model, accuracy:  7.10%


In [8]:
model.load_weights(checkpoint_path)

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4207 - sparse_categorical_accuracy: 0.8670
Restored model, accuracy: 86.70%


In [10]:
checkpoint_path = 'training_2/cp-{epoch:04d}.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size=32

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq = 5*batch_size)

model = create_model()

model.save_weights(checkpoint_path.format(epoch=0))

model.fit(train_images, train_labels,
          epochs=50,
          callbacks=[cp_callback],
          validation_data=(test_images, test_labels),
          verbose=0)


Epoch 00005: saving model to training_2\cp-0005.ckpt

Epoch 00010: saving model to training_2\cp-0010.ckpt

Epoch 00015: saving model to training_2\cp-0015.ckpt

Epoch 00020: saving model to training_2\cp-0020.ckpt

Epoch 00025: saving model to training_2\cp-0025.ckpt

Epoch 00030: saving model to training_2\cp-0030.ckpt

Epoch 00035: saving model to training_2\cp-0035.ckpt

Epoch 00040: saving model to training_2\cp-0040.ckpt

Epoch 00045: saving model to training_2\cp-0045.ckpt

Epoch 00050: saving model to training_2\cp-0050.ckpt


In [12]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2\\cp-0050.ckpt'

In [13]:
model = create_model()

model.load_weights(latest)

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4933 - sparse_categorical_accuracy: 0.8760
Restored model, accuracy: 87.60%


In [14]:
model.save_weights('./checkpoints/my_checkpoint')

model = create_model()

model.load_weights('./checkpoints/my_checkpoint')

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4933 - sparse_categorical_accuracy: 0.8760
Restored model, accuracy: 87.60%


In [15]:
model = create_model()
model.fit(train_images, train_labels, epochs=5)

model.save('saved_model/my_model')

Epoch 1/5
32/32 [==============================] - 0s 2ms/step - loss: 1.1852 - sparse_categorical_accuracy: 0.6640
Epoch 2/5
32/32 [==============================] - 0s 2ms/step - loss: 0.4194 - sparse_categorical_accuracy: 0.8800
Epoch 3/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2829 - sparse_categorical_accuracy: 0.9220
Epoch 4/5
32/32 [==============================] - 0s 2ms/step - loss: 0.1947 - sparse_categorical_accuracy: 0.9570
Epoch 5/5
32/32 [==============================] - 0s 2ms/step - loss: 0.1478 - sparse_categorical_accuracy: 0.9700
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: saved_model/my_model\assets


In [16]:
new_model = tf.keras.models.load_model('saved_model/my_model/')

new_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [17]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose = 2)
print('Restored model, acc: {:5.2f}%'.format(100*acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4190 - sparse_categorical_accuracy: 0.8630
Restored model, acc: 86.30%
(1000, 10)


In [23]:
model = create_model()
model.fit(train_images, train_labels, epochs=5)

model.save('my_model.h5')

Epoch 1/5
32/32 [==============================] - 0s 2ms/step - loss: 1.1968 - sparse_categorical_accuracy: 0.6620
Epoch 2/5
32/32 [==============================] - 0s 2ms/step - loss: 0.4441 - sparse_categorical_accuracy: 0.8780
Epoch 3/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2821 - sparse_categorical_accuracy: 0.9290
Epoch 4/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2057 - sparse_categorical_accuracy: 0.9550
Epoch 5/5
32/32 [==============================] - 0s 2ms/step - loss: 0.1681 - sparse_categorical_accuracy: 0.9640


In [24]:
new_model = tf.keras.models.load_model('my_model.h5')

new_model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_8 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [26]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored mode, accuracy: {:5.2f}%'.format(100*acc))


32/32 - 0s - loss: 0.4135 - sparse_categorical_accuracy: 0.8590
Restored mode, accuracy: 85.90%
